In [ ]:
import os, sys
os.environ['OPENBLAS_NUM_THREADS'] = '2' # to avoid a threads overflow error on cluster
script_dir = os.path.dirname(os.path.realpath(__file__))
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
import subprocess

from Bio import SeqIO
from Bio import PDB

%matplotlib inline
pd.options.display.max_colwidth = 1000

sys.path.append(f'{script_dir}/LA_scripts/software/')
from get_rmsds_functions import TMalign
from get_rmsds_functions import get_RMSD
import check_interactions_to_lig

from Bio.PDB.PDBParser import PDBParser
from Bio.SeqUtils import IUPACData

### Break mononer to multimers
// break original binders scaffolds into 2 monomers: monomer N and monomer C


In [ ]:
N=[1,88]
C=[91,256]
ref_pdb = f'{script_dir}/pdb/P1D7_B9_no_seq.pdb'
first,second =[],[]
name = os.path.basename(ref_pdb)[:-4] + '_4-8'
with open(ref_pdb,'r') as fp:
    for line in fp:
        if line.startswith('ATOM'):
            res_num = int(line[22:26])
            if res_num <= N[1] and res_num >= N[0]:
                first.append(line.rstrip())
            elif res_num <= C[1] and res_num >= C[0]:
                second.append(line.rstrip())
        elif line.startswith('HETATM') or line.startswith('TER') or line.startswith('CONECT'):
            first.append(line.rstrip())
            second.append(line.rstrip())

with open(f'{home_dir}/inputs_cut/{name}_N.pdb','w') as fp1:
    fp1.write('\n'.join(first))
with open(f'{home_dir}/inputs_cut/{name}_C.pdb','w') as fp2:
    fp2.write('\n'.join(second))


### ligMPNN
// sequence design with small molecule context

In [ ]:
pdb = f'{script_dir}/pdb/P1D7_B9_2_4_complex_tied_mpnn_no_seq.pdb'

In [ ]:
python = 'path/to/ligMPNN/python-envs/'
mpnn = f'path/to/ligMPNN/' #Dauparas 2023 https://doi.org/10.1101/2023.12.22.573103

cmd =f'{python} {mpnn} --out_folder ./ --num_seq_per_target 250 --pdb_path pdb/P1D7_B9_2_4_complex_tied_mpnn.pdb --omit_AAs C --pack_side_chains 1 --num_packs 1 --sampling_temp "0.1 0.2" --use_ligand 1 --tied_positions_jsonl pdb/tied_pos.jsonl --ligand_params_path pdb/CHD.params'

### AlphaFold each mononer
// af2 validate folding likelihood

In [ ]:
pdbs = glob.glob('/all/mpnn/generated/pdbs')
python = '/path/to/af2/python-envs'
run_af2 = f'{script_dir}/LA_scripts/toolkits/run_af2.py'
cmds = []
for pdb in pdbs:
    cmds.append(f"{python} {run_af2} -python {run_af2} -pdb {pdb}")
